In [ ]:
#МЕТОД КАНОНИЧЕСКИХ КОРРЕЛЯЦИЙ
import numpy as np
import pandas as pd

data  = pd.read_excel('LAB_DATA_MSM.xlsx', sheet_name = 'Показатели')

#columns_X = ['X2', 'X4', 'X5', 'X6', 'X7', 'X8']
columns_X = ['X1', 'X2', 'X3', 'X4', 'X5']
columns_Y = ['Y1', 'Y2', 'Y3']

X = np.array(data[columns_X])[1:,:].T
Y = np.array(data[columns_Y])[1:,:].T

mn = np.mean(X, axis = 1)
X = np.array([i-i.mean() for i in X])
Y = np.array([i-i.mean() for i in Y])
k = len(X)
n = X.shape[1]
X.shape

(5, 52)

In [ ]:
#Найдем оценки канонических переменных
#Найдем канонические веса
A_11 = np.cov(X)
A_22 = np.cov(Y)
A_12 = X@Y.T/(X.shape[1]-1)
A_21 = A_12.T
pd.DataFrame(A_12)

,0,1,2
0,-0.135901,-7.935196,0.008668
1,0.006740,1.404056,0.099368
2,0.059757,4.834662,-0.160036
3,0.075881,6.060097,0.158269
4,0.377337,33.862811,3.050748


In [ ]:
m = len(Y)
mat = np.linalg.inv(A_11)@A_12@np.linalg.inv(A_22)@A_21
eig_val, tetta = np.linalg.eig(mat)
ind = np.argsort(eig_val)[::-1]
tetta = tetta[ind].T

#Нормировка
for i in range(k):
  c = tetta[i].T@A_11@tetta[i]
  tetta[i] = tetta[i]/np.sqrt(c)


betta = np.zeros((m, m))
for i in range(m):
  betta[i] = np.linalg.inv(A_22)@A_21@tetta[i]/np.sqrt(eig_val[i])


In [ ]:
tetta

array([[  2.01463276,   5.24299477,   1.89446234,  -0.22691388,
          1.47129193],
       [-10.39846329,  -5.98172063,  -0.82562365,   1.23458471,
         -0.3743823 ],
       [ -7.63210819,  -9.14789034,  -7.254845  ,  -4.68149546,
          0.31248734],
       [ -4.06450305,  14.34401144,  -0.93607687,  -7.15119737,
         -0.13377069],
       [ -1.18279146,   7.05624602,  -4.20897463,   3.7649144 ,
         -0.64258753]])

In [ ]:
#Оценки канонических корреляций
ro = np.sqrt(eig_val[:m])
ro

array([0.85240983, 0.66147094, 0.1774935 ])

In [ ]:
from scipy.stats.distributions import chi2
#Проверка значимости канонических переменных
def statistic(p, m, n, k, eig_val):
  l = sum(np.log(1-i) for i in eig_val[p-1:m])
  etta = -(n-1-1/2*(m+k+1))*l
  nu = (k-p+1)*(m-p+1)
  chi_2 = chi2.ppf(0.9, df=nu)
  print(f'Для p={p}: критическое значение = {chi_2:.4f}, статистика = {etta:.4f}')
  if etta<chi_2:
    print('Гипотеза H0 принимается')
  else:
    print('Гипотеза H0 отвергается')

statistic(1, m, n, k,  eig_val)


Для p=1: критическое значение = 22.3071, статистика = 88.5491
Гипотеза H0 отвергается


In [ ]:
statistic(2, m, n, k, eig_val)

Для p=2: критическое значение = 13.3616, статистика = 28.2466
Гипотеза H0 отвергается


In [ ]:
statistic(3, m, n, k, eig_val)

Для p=3: критическое значение = 6.2514, статистика = 1.4885
Гипотеза H0 принимается


In [ ]:
#Запишем канонические переменные
v = betta[:2]@Y
u = tetta[:2]@X
#Найдем корреляции между каноническими переменными и переменными из множеств X, Y
#(координаты векторов канонических нагрузок)
cov_X_U1_formula = A_11 @ tetta[ 0]
cov_Y_V1_formula = A_22 @ betta[ 0]
cov_X_U2_formula = A_11 @ tetta[ 1]
cov_Y_V2_formula = A_22 @ betta[ 1]
pd.DataFrame(cov_X_U1_formula)

,0
0,-0.030391
1,0.023582
2,-0.003894
3,0.052916
4,0.650431


In [ ]:
pd.DataFrame(cov_Y_V1_formula)

,0
0,0.484169
1,57.461793
2,5.499580


In [ ]:
#Величина объясняемой(извлеченной) канонической переменной дисперсии для соответствующего множества
def length(X):
  return np.sqrt(np.sum(X**2))
V_u1 = length(cov_X_U1_formula)**2
V_u2 = length(cov_X_U2_formula)**2
V_v1 = length(cov_Y_V1_formula)**2
V_v2 = length(cov_Y_V2_formula)**2
V_u1, V_u2,V_v1, V_v2

(np.float64(0.42735511197425347),
 np.float64(0.019190171502393334),
 np.float64(3332.3373993506866),
 np.float64(9643.54247943938))

In [ ]:
#Доли суммарных дисперсий исходных признаков, объясняемые  p=2 каноническими переменными
D1 = (V_u1+V_u2)/np.trace(A_11)
D2 = (V_v1+V_v2)/np.trace(A_22)
D1, D2

(np.float64(0.8458560755112381), np.float64(0.9386366803117189))

In [ ]:
#Избыточность первого множества переменных при заданном втором множестве
redundancy1 = (V_u1*ro[0]**2+V_u2*ro[1]**2)/np.trace(A_11)
redundancy2 = (V_v1*ro[0]**2+V_v2*ro[1]**2)/np.trace(A_22)
redundancy1, redundancy2

(np.float64(0.6040937359652102), np.float64(0.48037267369155545))